In [33]:
import os
from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv()
print("Service Account KEY:", os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

Service Account KEY: /Users/ywang/.config/gcloud/pacific-card-374422-5e253ea4302a.json


In [2]:
client = bigquery.Client()

In [14]:
def qry_bg(query:str):
    query_job = client.query(query)
    for row in query_job.result():
        print(f"Answer is: {row[0]}")

In [34]:
create_ext_table_qry ="""
                    CREATE OR REPLACE EXTERNAL TABLE `pacific-card-374422.dezoomcamp.external_fhv_tripdata`
                    OPTIONS (
                    format = 'PARQUET',
                    uris = ['gs://prefect-de-zoomcamp-2/data/fhv/fhv_tripdata_2019-*.parquet']
                    )
                    """
                    
create_table_qry = """
                    create or replace table `pacific-card-374422.dezoomcamp.fhv_tripdata` as  
                    select * from `pacific-card-374422.dezoomcamp.external_fhv_tripdata`; 
                    """

In [35]:
# Create external table
client.query(create_ext_table_qry).result()

In [36]:
# Create materialize table
client.query(create_table_qry).result()

In [37]:
# Question 1
q1_qry="""select count(*) from pacific-card-374422.dezoomcamp.external_fhv_tripdata;"""
qry_bg(q1_qry)

Answer is: 43244696


In [38]:
# Question 2
q2_p1_qry="""select count(distinct(affiliated_base_number)) from pacific-card-374422.dezoomcamp.external_fhv_tripdata;"""
qry_bg(q2_p1_qry)

Answer is: 3163


In [39]:
# Question 2 Bytes processed 317.94 MB
q2_p2_qry="""select count(distinct(affiliated_base_number)) from pacific-card-374422.dezoomcamp.fhv_tripdata;"""
qry_bg(q2_p2_qry)

Answer is: 3163


In [40]:
# Question 3
q3_qry="""select count(*) from pacific-card-374422.dezoomcamp.fhv_tripdata
            where PUlocationID is NULL and DOlocationID is NULL ;"""
qry_bg(q3_qry)

Answer is: 717748


# Question 4

Partition by pickup_datetime Cluster on affiliated_base_number

In [42]:
# Question 5
q5_qry="""CREATE OR REPLACE TABLE `pacific-card-374422.dezoomcamp.fhv_tripdata_pc`
        PARTITION BY DATE(pickup_datetime)                                         
        CLUSTER BY affiliated_base_number AS                                                        
        SELECT * FROM `pacific-card-374422.dezoomcamp.fhv_tripdata`;"""
client.query(q5_qry).result()

In [45]:
# Question 5 Bytes processed 647.87 MB
q5_normal_qry="""                                                        
        SELECT COUNT(DISTINCT affiliated_base_number) as affiliated_base_numbers FROM `pacific-card-374422.dezoomcamp.fhv_tripdata`
        WHERE DATE(pickup_datetime) BETWEEN '2019-03-01' AND '2019-03-31';"""
qry_bg(q5_normal_qry)

Answer is: 724


In [46]:
# Question 5 Bytes processed 23.05 MB
q5_pc_qry="""                                                        
        SELECT COUNT(DISTINCT affiliated_base_number) as affiliated_base_numbers FROM `pacific-card-374422.dezoomcamp.fhv_tripdata_pc`
        WHERE DATE(pickup_datetime) BETWEEN '2019-03-01' AND '2019-03-31';"""
qry_bg(q5_pc_qry)

Answer is: 724


# Question 6
- GCP Bucket

In [ ]:
# Question 7
- False